In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
dataset = dbutils.widgets.get("dataset")

model_name = dbutils.widgets.get("model_name")
production_alias = dbutils.widgets.get("production_alias")

endpoint_name = dbutils.widgets.get("endpoint_name")

dataset_path = f"{catalog}.{schema}.{dataset}"
model_path = f"{catalog}.{schema}.{model_name}"
print(f"Iris Dataset Path: {dataset_path}")
print(f"Iris Model Path: {model_path}")

In [0]:
import mlflow
from mlflow.deployments import get_deploy_client
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

mlflow.set_registry_uri("databricks-uc")
client = get_deploy_client("databricks")

try:
    endpoint = client.create_endpoint(
    name=f"{endpoint_name}",
    config={
        "served_entities": [
            {
                "entity_name": f"{catalog}.{schema}.{model_name}",
                "entity_version": "1",
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ],
        "traffic_config": {
            "routes": [
                {
                    "served_model_name": f"{model_name}-1",
                    "traffic_percentage": 100
                }
            ]
        }
    }
    )
except Exception:
    print("Endpoint already exists")

In [0]:
import json
import pandas as pd

# Prepare a sample Iris row (sepal_length, sepal_width, petal_length, petal_width)
sample = pd.DataFrame([
    {"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2}
])

# Invoke endpoint
# "inputs" payload supports 'dataframe_records' for pandas-style records
payload = {
    "inputs": [
        {
            "name": "inputs",
            "shape": [len(sample), len(sample.columns)],
            "datatype": "FP32",
            "dataframe_records": sample.to_dict(orient="records")
        }
    ]
}

invocation = w.serving_endpoints.query(name=endpoint_name, dataframe_records=payload["inputs"][0]["dataframe_records"])  # type: ignore

# Best-effort parse of common MLflow serving output formats
try:
    # Newer SDKs return structured QueryEndpointResponse
    preds = getattr(invocation, "predictions", None)
    if preds is None and isinstance(invocation, dict):
        preds = invocation.get("predictions")
    print("Predictions:", preds[0])
except Exception as e:
    print("Could not parse predictions:", e)
    print(invocation)
